# Coze保姆级教程

coze是字节跳动旗下的零代码大模型Agent定制平台~

具有以下主要功能特点：

* 1，零成本发布: 可非常简单发布自己定制的bot到 豆包，公众后订阅号，微信客服，掘金等平台。

* 2，多种模型选择： 包括 kimi，豆包，baichuan4, 通义千问，MiniMax，GLM4等均可使用... 

* 3，丰富的插件工具：必应搜索，文生图，图片理解，图片OCR，wolfram alpha，网页解析，文档解析，知乎新浪热榜，搜索猎聘岗位等等

* 4，强大的知识库：支持文档，表格，图片注入 知识库数据。

* 5，自定义插件工具：会编程的用户可以使用Python和javascript函数 设计自己的插件工具。

* 6，自定义工作流：通过串联组合  插件工具， 自定义函数，配置了特定prompt的大模型等 设计出具有稳定输出的工作流，比如将 文章故事 转换成视频， 根据提示词 绘制漫画绘本 等等。

* 7，自定义bot：可以选择一个模型，配置好prompt，添加各种工具插件，知识库，工作流，定制自己的bot。

* 8，免费API接口调用: 可以通过http请求调用自己的bot，目前每个账号一天可以调用3000次。


## 〇，注册coze 

coze网址是：https://www.coze.cn/。也可以搜索扣子进入页面去注册。用手机号注册即可。

注册后进入了coze，左侧会是一个这样的页面。

注意圈出来的这两个地方。

蓝色的是官方详细的coze搭建智能体创建工作流自定义插件等的教程文档。

红色的是API接口的使用文档，不是很好找，遇到不会的问题点进去看文档。


![](data/扣子界面.png)

## 一，定制Agent

我定制了一个 bot，叫做 梦中情炉。

这个agent主要是为了调用coze丰富的插件能力，是一个无情的插件调用工具人。

bot页面地址为：https://www.coze.cn/store/bot/7380551805660430372


![](data/编辑界面.png)

1，首先选择一个模型，我选择的是 moonshot，也就是kimi。

2，写这个bot的人设与恢复逻辑。

可以先写一个简单粗糙的prompt。

比如我开始写的是：你是一个全能型agent，擅长分析问题，通过选择和组合工具获取信息并解决问题。

然后点击右侧的自动优化，再反复人工调整，反复自动优化。


3，接着配置一些插件工具和工作流。

我给梦中情炉堆了许多工具：

* (1)，必应文字搜索，图片搜索，头条搜索，国内股票查询
* (2)，知乎热榜，掘金热榜，微博热榜，头条新闻查询
* (3)，猎聘 岗位查询
* (4)，python代码执行器，wolfram alpha
* (5)，图片理解，图片OCR识别，文生图。

简单起见，没有搞工作流。工作流相当于你自己整合大模型，和各种工具以便产出一个的输出。

4，接着可以配置一些表格或者文档弄成知识库。

简单起见，没有搞知识库。

5，最后，可以配置一下开场白和预设问题。


![](data/开场白.png)



## 二， 公众号接入

配置好的话，可以在最右边一栏调试一下，然后点击右上角的发布就可以了。

![](data/调试和发布.png)


发布的时候配置一下公众号的APPID，扫码确认一下就可以了。

公众号的APPID在公众号的后台的这个地方获取，没有公众号的话可以去注册一个。

![](./data/获取APPID.png)

配置好并发布后，就可以去微信公众号对话体验了。

我发布在了我的一个小号：一个有毅力的吃货。大家可以去试试看，有时候可能略微有些慢。

这个号因为设置了很多关键词给大家回复各种代码资料，如果接了coze的功能，大家就不好找这些代码资料了。

![](./data/聊天截图.jpg)

## 三，Python代码调用 

除了在公众号会话中接入，我们还可以使用Python代码接入。

使用Python接口可以更加方便灵活地使用。

使用Python代码接入的话，要做一些准备工作。

1，发布的使用需要选上最下面这个Bots AS API发布。

![](data/授权页面.png) 


2，然后去获取两个信息，一个是bot_id, 另一个是个人秘钥api_token。

bot_id就在这个bot的编辑页面的后面的数字。

个人秘钥api_token需要在这个网址创建。

https://www.coze.cn/open/api



![](./data/bot_id.png)






原始的API接口文档是使用bash curl命令调用的。

我把它用Python代码，一个Coze类封装了一遍，更加好用一些。

In [1]:
import sys 
import requests,json 
from IPython.display import display,clear_output,Markdown

API_TOKEN = 'pat_xxxxxxxxx'
BOT_ID = "7362845197946257419"

class Coze:
    def __init__(self,
                 bot_id= BOT_ID,
                 api_token = API_TOKEN,
                 max_chat_rounds=20,
                 stream=True,
                 history=None
                ):
        self.bot_id = bot_id
        self.api_token = api_token 
        self.history = [] if history is None else history
        self.max_chat_rounds = max_chat_rounds
        self.stream = stream
        self.url = 'https://api.coze.cn/open_api/v2/chat'
        self.headers = {
            'Authorization': f'Bearer {self.api_token}',
            'Content-Type': 'application/json','Accept': '*/*',
            'Host': 'api.coze.cn','Connection': 'keep-alive'
        }
        try:
            self.register_magic() 
            response = self('你好')
            if not self.stream:
                print(response)
            print('register magic %%chat sucessed ...',file=sys.stderr)
            self.history = self.history[:-1]
        except Exception as err:
            print('register magic %%chat failed ...',file=sys.stderr)
            print(err)
             
    @classmethod
    def build_messages(cls,history=None):
        messages = []
        history = history if history else [] 
        for prompt,response in history:
            pair = [{"role": "user", "content": prompt,
                     "content_type":"text"},
                {"role": "assistant", "content": response}]
            messages.extend(pair)
        return messages

    @staticmethod
    def get_response(messages):
        clear_output(wait=True)
        dfmsg = pd.DataFrame(messages)
        dftool = dfmsg.loc[dfmsg['type']=='function_call']
        for content in dftool['content']:
            info =json.loads(content)
            s = 'call function: '+str(info['name'])+'; args ='+str(info['arguments'])
            print(s,file=sys.stderr)
        dfans = dfmsg.loc[dfmsg['type']=='answer']
        if len(dfans)>0:
            response = ''.join(dfans['content'].tolist())
        else:
            response = ''
        display(Markdown(response))
        return response

    def chat(self,query,stream=False):
        data = {
            "conversation_id": "123",
            "bot_id": self.bot_id,
            "user": "一个有毅力的吃货",
            "query": query,
            "stream": stream,
            "chat_history": self.build_messages(self.history)
        }
        json_data = json.dumps(data)
        result = requests.post(self.url, headers=self.headers, 
            data=json_data, stream=data["stream"])
    
        if not data["stream"] and result.status_code == 200:
            dic = json.loads(result.content.decode('utf-8'))
            response = self.get_response(dic['messages'])
                
        elif data['stream'] and result.status_code == 200:
            messages = []
            for line in result.iter_lines():
                if not line:
                    continue
                try:
                    line = line.decode('utf-8')
                    line = line[5:] if line.startswith('data:') else line
                    dic = json.loads(line)
                    if dic['event']=='message':
                        messages.append(dic['message'])
                    response = self.get_response(messages)
                except Exception as err:
                    print(err)
                    break 
        else:
            print(f"request failed, status code: {result.status_code}")
        result.close()
        return response
        
        
    def __call__(self,query):
        from IPython.display import display,clear_output 
        len_his = len(self.history)
        if len_his>=self.max_chat_rounds+1:
            self.history = self.history[len_his-self.max_chat_rounds:]
        response = self.chat(query,stream=self.stream) 
        self.history.append((query,response))
        return response 

    def register_magic(self):
        import IPython
        from IPython.core.magic import (Magics, magics_class, line_magic,
                                        cell_magic, line_cell_magic)
        @magics_class
        class ChatMagics(Magics):
            def __init__(self,shell, pipe):
                super().__init__(shell)
                self.pipe = pipe

            @line_cell_magic
            def chat(self, line, cell=None):
                "Magic that works both as %chat and as %%chat"
                if cell is None:
                    return self.pipe(line)
                else:
                    self.pipe(cell)       
        ipython = IPython.get_ipython()
        magic = ChatMagics(ipython,self)
        ipython.register_magics(magic)
        

In [15]:
chat = Coze(max_chat_rounds=10)

您好！有什么我可以帮助您的吗？如果您有任何问题或者需要协助，随时告诉我。

register magic %%chat sucessed ...


In [16]:
%%chat
广州有什么最新的算法岗位吗？提供一下原始岗位链接。

您想要查找广州的算法岗位。请问您是否有特定的公司类型或公司名称，以及对薪水的范围有具体的要求吗？

In [17]:
%%chat
互联网行业就可以，薪资20k以上的。

call function: liepin-job_recommendation; args ={'address': '广州', 'jobName': '算法', 'salaryCap': '200000', 'salaryKind': '年薪'}


广州地区当前最新的算法岗位信息如下：

1. 岗位名称 **算法工程师**，来自 **深海智人** 公司。薪资范围为17-25k。详细信息请参阅 [岗位链接](https://www.liepin.com/job/1967869471.shtml?mscid=so_pc_002)。

2. 岗位名称 **自动驾驶.控制策略与算法开发(J11430)**，来自 **东风日产广州** 公司。薪资范围为16-24k。详细信息请参阅 [岗位链接](https://www.liepin.com/job/1966930237.shtml?mscid=so_pc_002)。

3. 岗位名称 **自动驾驶.传感器融合策略与算法开发(J11429)**，同样来自 **东风日产广州** 公司。薪资也位于16-24k的范围。详细信息请参阅 [岗位链接](https://www.liepin.com/job/1966929783.shtml?mscid=so_pc_002)。

4. 来自 **东风日产广州** 的 **车联网智能导航应用开发(J11115)** 岗位信息也可查看 [岗位链接](https://www.liepin.com/job/1947687947.shtml?mscid=so_pc_002)。

5. 另有一个未明确公司名称的大型整车制造公司提供的 **自动驾驶方向开发** 岗位，薪资15-20k，详情见 [岗位链接](https://www.liepin.com/a/55137003.shtml?mscid=so_pc_002)。

这些岗位覆盖了车辆自动化和智能控制的各个方面，并来自不同规模的公司。具体详情可通过岗位链接进行查询。

In [18]:
chat.history 

[('广州有什么最新的算法岗位吗？提供一下原始岗位链接\n',
  '您想要查找广州的算法岗位。请问您是否有特定的公司类型或公司名称，以及对薪水的范围有具体的要求吗？'),
 ('互联网行业就可以，薪资20k以上的。\n',
  '广州地区当前最新的算法岗位信息如下：\n\n1. 岗位名称 **算法工程师**，来自 **深海智人** 公司。薪资范围为17-25k。详细信息请参阅 [岗位链接](https://www.liepin.com/job/1967869471.shtml?mscid=so_pc_002)。\n\n2. 岗位名称 **自动驾驶.控制策略与算法开发(J11430)**，来自 **东风日产广州** 公司。薪资范围为16-24k。详细信息请参阅 [岗位链接](https://www.liepin.com/job/1966930237.shtml?mscid=so_pc_002)。\n\n3. 岗位名称 **自动驾驶.传感器融合策略与算法开发(J11429)**，同样来自 **东风日产广州** 公司。薪资也位于16-24k的范围。详细信息请参阅 [岗位链接](https://www.liepin.com/job/1966929783.shtml?mscid=so_pc_002)。\n\n4. 来自 **东风日产广州** 的 **车联网智能导航应用开发(J11115)** 岗位信息也可查看 [岗位链接](https://www.liepin.com/job/1947687947.shtml?mscid=so_pc_002)。\n\n5. 另有一个未明确公司名称的大型整车制造公司提供的 **自动驾驶方向开发** 岗位，薪资15-20k，详情见 [岗位链接](https://www.liepin.com/a/55137003.shtml?mscid=so_pc_002)。\n\n这些岗位覆盖了车辆自动化和智能控制的各个方面，并来自不同规模的公司。具体详情可通过岗位链接进行查询。')]